In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [10]:
# configurations
n_steps = 96*88
input_dims = 1
n_classes = 2 

# model config
cell_type = "RNN"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20] * 9
dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = 4
testing_step = 5000
display_step = 100

# permutation seed 
seed = 92916

In [11]:
X = np.load("X_0.npy").astype(np.float32)
Y = np.load("Y_0.npy").astype(np.int32)
train_set = np.load("train_0.p")
valid_set = np.load("valid_0.p")
test_set = np.load("test_0.p")
train_data = X[len(valid_set):len(valid_set)+len(train_set)]
train_labels = Y[len(valid_set):len(valid_set)+len(train_set)]
eval_data = X[:len(valid_set)]
eval_labels = Y[:len(valid_set)]
print(train_data.shape)
print(train_labels.shape)
print(eval_data.shape)
print(eval_labels.shape)
print(train_labels)

if 'seed' in globals():
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)

(13907, 192, 88, 1)
(13907,)
(2291, 192, 88, 1)
(2291,)
[0 1 0 ..., 0 0 1]


In [13]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
print("building cost")
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
print("building optimizer")
optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost, global_step=global_step)

# evaluation model
print("building correct_pred")
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
print("building accuracy")
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with RNN cells
Building layer: multi_dRNN_dilation_1, input length: 8448, dilation rate: 1, input dim: 1.
=====> Input length for sub-RNN: 8448
Building layer: multi_dRNN_dilation_2, input length: 8448, dilation rate: 2, input dim: 20.
=====> Input length for sub-RNN: 4224
Building layer: multi_dRNN_dilation_4, input length: 8448, dilation rate: 4, input dim: 20.
=====> Input length for sub-RNN: 2112
Building layer: multi_dRNN_dilation_8, input length: 8448, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 1056
Building layer: multi_dRNN_dilation_16, input length: 8448, dilation rate: 16, input dim: 20.
=====> Input length for sub-RNN: 528
Building layer: multi_dRNN_dilation_32, input length: 8448, dilation rate: 32, input dim: 20.
=====> Input length for sub-RNN: 264
Building layer: multi_dRNN_dilation_64, input length: 8448, dilation rate: 64, input dim: 20.
=====> Input length for sub-RNN: 132
Building layer: multi_dRNN_dilation_128, input length

In [14]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
step = 0
train_results = []
validation_results = []
test_results = []

index_in_epoch = 0
num_examples = len(train_data)
print(train_data.shape)
epochs_completed = 0
print("epoch 0")

while step < training_iters:
#     batch_x, batch_y = mnist.train.next_batch(batch_size)
    print(step)
    start = index_in_epoch
    index_in_epoch += batch_size
    if index_in_epoch > num_examples:
        epochs_completed += 1
        print("epoch", epochs_completed)
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_data = train_data[perm]
        train_labels = train_labels[perm]
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    
    batch_x = train_data[start:end]
    batch_y = train_labels[start:end]
    print(batch_x.shape)
    print(batch_y.shape)
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])
    print(batch_x.shape)
    batch_y = np.eye(n_classes)[batch_y]
    print(batch_y.shape)
    print("---")
    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # permute the data
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_))
    step += 1

(13907, 192, 88, 1)
epoch 0
0
(128, 192, 88, 1)
(128,)


ValueError: cannot reshape array of size 2162688 into shape (128,8448,1)

In [ ]:
# validation performance
batch_x = mnist.validation.images
batch_y = mnist.validation.labels

# permute the data
batch_x = batch_x[:, idx_permute]        
batch_x = batch_x.reshape([-1, n_steps, input_dims])
feed_dict = {
    x : batch_x,
    y : batch_y
}
cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
validation_results.append((step_, cost_, accuracy__))

# test performance
batch_x = mnist.test.images
batch_y = mnist.test.labels
batch_x = batch_x[:, idx_permute]        
batch_x = batch_x.reshape([-1, n_steps, input_dims])
feed_dict = {
    x : batch_x,
    y : batch_y
}
cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
test_results.append((step_, cost_, accuracy_))        
print("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
+ ", Testing Accuarcy: " + "{:.6f}".format(accuracy_))